In [ ]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import faiss
import os, io, pickle
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    SpacyTextSplitter
)
from langchain.document_loaders import PyPDFDirectoryLoader  # a class for loading PDF documents from a directory
from langchain.vectorstores import FAISS
import numpy as np
import pandas as pd
import tiktoken
from sentence_transformers import SentenceTransformer
import shutil
import statistics
#import fitz  # PyMuPDF
#from langchain.embeddings import OpenAIEmbeddings
#from langchain_community.vectorstores import FAISS
#from langchain_community.embeddings import OpenAIEmbeddings
#from langchain.llms import OpenAI

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate_google():
    creds = None
    if os.path.exists("token"):
        with open("token", "rb") as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials3.json", SCOPES)
            creds = flow.run_local_server(port=0)
        with open("token", "wb") as token:
            pickle.dump(creds, token)
    
    return build("drive", "v3", credentials=creds)

def download_pdf(file_id, output_path):
    service = authenticate_google()
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(output_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Téléchargement : {int(status.progress() * 100)}%")
    print(f"Fichier téléchargé : {output_path}")




In [5]:
#file_id = "1AHE1lXi_kyrtRw31qEGJ7OYE9EO8kfGe"
download_pdf("1AHE1lXi_kyrtRw31qEGJ7OYE9EO8kfGe", "Histoire_CM1.pdf")
download_pdf("1cGcGr-YYfIxUzYwmHHcxcp7kL-fuppqA", "histoire-CM1-emanuel.pdf")

Téléchargement : 100%
Fichier téléchargé : Histoire_CM1.pdf
Téléchargement : 100%
Fichier téléchargé : histoire-CM1-emanuel.pdf


In [6]:
# Fonction pour estimer le nombre de tokens d’un texte
def count_tokens(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

In [9]:
# Charger ton PDF
loader = PyPDFLoader("Histoire_CM1.pdf",
                     mode = "page", # Extract the PDF by page. Each page is extracted as a langchain Document object
                     # mode = "single" # PyPDFLoader will split the PDF as a single text flow
                     )
docs = loader.load()

In [ ]:
# Importer plusieurs PDF au moyen de PyPDFDirectoryLoader
# Par défaut, importe page par page 
loader = PyPDFDirectoryLoader(
    path = "./",
    glob = "**/[!.]*.pdf",
    extract_images = False,
)

docs = loader.load()

In [10]:
docs

[Document(metadata={'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2018-07-23T15:58:49+02:00', 'title': 'Leçons d’histoire  CM1', 'author': 'Saleur', 'moddate': '2018-07-23T15:58:49+02:00', 'source': 'Histoire_CM1.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content='LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les

In [11]:
print("Nombre de pages :", len(docs))
print("Métadonnées de la première page :", docs[0].metadata)

Nombre de pages : 12
Métadonnées de la première page : {'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2018-07-23T15:58:49+02:00', 'title': 'Leçons d’histoire  CM1', 'author': 'Saleur', 'moddate': '2018-07-23T15:58:49+02:00', 'source': 'Histoire_CM1.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}


In [12]:
# Si option mode = "single" 
#full_text = docs[0].page_content
#full_text

In [13]:
# Si mode = "page", pour concaténer tout le contenu du PDF en un seul texte brut
full_text = "\n".join([page.page_content for page in docs])
full_text

'LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsait qu’elle a été peinte il y a environ 17000 ans.  \n \n*En 1994, Jean -Marie Chauvet découvre une autre grotte pei

CharacterTextSplitter : simple et rapide à mettre en oeuvre mais rigide et ne garde pas la cohérence du contexte (coupe mécanique basée sur un séparateur, donc des concepts liés peuvent se retrouver séparés dans plusieurs chunks)
RecursiveCharacterTextSplitter: découpe le texte (par paragraphes, phrases, mots) de façon itérative jusqu'à ce que le chunk atteint la limite de taille imposée. Respect de la taille fixée, mais risque de fragmentation de la cohérence du texte
SpacyTextSplitter:
SemanticSplitter: découpage en chunks de façon sémantique. Utilise les embeddings pour calculer la similarité entre phrases ou paragraphes. Les clusters sont créés en fonction de critères sémantiques (ex : cosine similarity). Produit des chunks avec une grande cohérence sémantique, ce qui semble idéal pour un RAG. Stratégie adaptative au contenu (ne suit pas un critère rigide de taille de caractères ou tokens). Moins performant en termes de temps d'exécution et consommateur de ressources (besoin de générer des embeddings et des calculs de similarité) et sensibilité au seuil (requiert tuning pour trouver l'équilibre entre la taille du chunk et sa cohérence)

In [ ]:
!python -m spacy download fr_core_news_sm

# Définir les splitters à tester
splitters = {
    "RecursiveCharacterTextSplitter": RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50),
    "RecursiveCharacterTextSplitter2" : RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, separators = ["\n\n", "\n", " ", ""]),
    "CharacterTextSplitter": CharacterTextSplitter(separator="\n \n", chunk_size=500, chunk_overlap=50),
    "SpacyTextSplitter": SpacyTextSplitter(pipeline="fr_core_news_sm", chunk_size=500)
}

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 16.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [16]:
# Tester les splitters
for name, splitter in splitters.items():
    chunks = splitter.split_text(full_text)
    token_counts = [count_tokens(chunk) for chunk in chunks]

    print(f"\n🧩 {name}")
    print(f" - Nombre de chunks : {len(chunks)}")
    print(f" - Tokens/chunk (moy) : {round(statistics.mean(token_counts))}")
    print(f" - Tokens max : {max(token_counts)}")
    print(f" - Aperçu du 1er chunk :\n{chunks[0]}\n")
    #print(f" - Aperçu du 2e chunk :\n{chunks[1]}\n")
    #print(f" - Aperçu du 3e chunk :\n{chunks[2]}\n")



🧩 RecursiveCharacterTextSplitter
 - Nombre de chunks : 27
 - Tokens/chunk (moy) : 144
 - Tokens max : 161
 - Aperçu du 1er chunk :
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.


🧩 CharacterTextSplitter
 - Nombre de chunks : 30
 - Tokens/chunk (moy) : 130
 - Tokens max : 162
 - Aperçu du 1er chunk :
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des ob

In [ ]:
# Chunking sémantique
import spacy
from sentence_transformers import SentenceTransformer, util
nlp = spacy.load("fr_core_web_sm")
model = SentenceTransformer("all-MiniLM-L6-v2")

def semantic_embedding_chunk(text, threshold = 0.75):
    """
    Découpe le texte en chunks en utilisant des embeddings de phrase
    Utilise spaCy pour la segmentation en phrase et SentenceTranformer pour la création d'embeddings

    Args:
        text (_type_): le texte à découper en chunks
        threshold (float, optional): seuil de cosine similarity au-delà duquel on ajoute une phrase au chunk courant. Defaults to 0.75.
    
    Return : une liste de chunks (chaque chunk est une chaîne de caractères)
    """
    
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    current_chunk_sentences = []
    current_chunk_embedding = None
    
    for sentence in sentences:
        #Generate embedding for the current sentence
        sentence_embedding = model.encode(sentence, convert_to_tensor = True)
        
        # If starting a new chunk, initialize it with the current sentence
        if current_chunk_embedding is None:
            current_chunk_sentences = [sentence]
            current_chunk_embedding = sentence_embedding
        else:
            # Compute cosine similarity betwwen current sentence and the chunk embedding
            sim_score = util.cos_sim(sentence_embedding, current_chunk_embedding)
            if sim_score.item() >= threshold:
                # Add sentence to the current chunk and update the chunk's average embedding
                ...

Pour pouvoir dérouler la suite, on va retenir dans un premier temps la stratégie de chunking basée sur le RecursiveCharacterSplitter sans tokenization.

In [17]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# Quelle différence entre split_text et split_documents ?
# split_text renvoie une liste de strings correspondant à la liste des morceaux de texte
# split_documents renvoie une liste de "Langchain Documents" (il semble splitter les chunks par page)
# Le nombre de chunks produits est différent : 27 avec split_text, 31 avec split_documents
chunks = splitter.split_text(full_text) 
#chunks = splitter.split_documents(docs)

In [18]:
print(type(chunks))
print(len(chunks))

<class 'list'>
27


In [19]:
chunks

['LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.',
 'l’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsait qu’elle a été peinte il y a environ 17000 ans.',
 '*En 1994, Jean -Marie Chauvet découvre une a

In [20]:
print("\n--- Statistiques sur les chunks du document ---")
print(f" - Nombre de chunks : {len(chunks)}")
print(f" - Tokens/chunk (moy) : {round(statistics.mean(token_counts))}")
print(f" - Tokens max : {max(token_counts)}")
print(f" - Aperçu du 1er chunk :\n{chunks[0]}\n")
print(f" - Aperçu du 2e chunk :\n{chunks[1]}\n")
print(f" - Aperçu du 3e chunk :\n{chunks[2]}\n")


--- Statistiques sur les chunks du document ---
 - Nombre de chunks : 27
 - Tokens/chunk (moy) : 129
 - Tokens max : 160
 - Aperçu du 1er chunk :
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.

 - Aperçu du 2e chunk :
l’Histoire.  
*L’Histoire de France est divisée en 5 périodes : 
l’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème 
siècle – le XXème siècle. 
 
 
 
 
2) Des traces du passé : les grottes ornées 
 
*En 1940, 4 enfants découvrent une grotte recouverte 
de peintures  : des taureaux, des cerfs, des chevaux…  : 
la grotte de Lascaux.  En datant

In [21]:
# Nombre de caractères par chunk
print([len(chunk) for chunk in chunks])

# On vérifie que le nombre de caractères est inférieur au chunk_size spécifié (500) 

[474, 443, 442, 481, 486, 472, 443, 489, 444, 491, 445, 456, 453, 445, 451, 450, 498, 490, 494, 485, 474, 449, 461, 448, 435, 463, 208]


# Création des embeddings

In [40]:
# Charger le modèle d'embeddings ici avec SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

# Générer les embeddings
embeddings = model.encode(chunks)

# Taille des embeddings
print("Taille des embeddings : ", embeddings.shape)

# Affichage clair
for i, (chunk, emb) in enumerate(zip(chunks, embeddings)):
    print("="*60)
    print(f"Chunk {i+1}:")
    print(chunk)
    print(f"\n Embedding (taille {len(emb)}):")
    print(np.round(emb[:10], 3))  # Affiche les 10 premières valeurs, arrondies pour lisibilité
    print("="*60)

Taille des embeddings :  (27, 384)
Chunk 1:
LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  
 
 
1)  Qu’est-ce que l’Histoire ? 
 
*L’Histoire est l’étude de notre passé  pour mieux 
comprendre notre vie aujourd’hui. 
 
 *Pour découvrir notre passé, les historiens font des fouilles 
archéologiques, étudient des objets, des documents, des récits…  
*Ils représentent le temps par une ligne graduée  : c’est la frise 
chronologique.  
 
*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient 
l’Histoire.

 Embedding (taille 384):
[-0.024  0.15   0.059 -0.052 -0.017  0.151 -0.052  0.037 -0.035  0.003]
Chunk 2:
l’Histoire.  
*L’Histoire de France est divisée en 5 périodes : 
l’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème 
siècle – le XXème siècle. 
 
 
 
 
2) Des traces du passé : les grottes ornées 
 
*En 1940, 4 enfants découvrent une grotte recouverte 
de peintures  : des taureaux, des cerfs, des chevaux…  : 
la grotte de Lascaux.  En datant les objets trouvés d

# FAISS VectorDB

IndexFlat2 calcule la distance euclidienne entre des vecteurs. 

In [41]:
# Création de l'index FAISS
dim = embeddings.shape[1]  # 384 pour MiniLM
index = faiss.IndexFlatL2(dim)  # Index basé sur la distance euclidienne
index.add(np.array(embeddings))  # Ajout des vecteurs

print("Index FAISS créé avec", index.ntotal, "vecteurs.")


Index FAISS créé avec 27 vecteurs.


### Test : récupérer les N chunks les plus similaires à la requête

cf https://lajavaness.medium.com/indexation-et-recherche-avec-faiss-c7675c42abb9

On effectue une recherche avec index.search(xq, k) avec

* xq notre requête. Il peut s’agir d’un seul vecteur ou d’un array de plusieurs vecteurs.
* k est le nombre de résultats par vecteur à renvoyer.

La méthode search() renvoie deux objets :

* le résultat de notre recherche : une matrice I de taille (n, k) où n est le nombre de vecteurs dans notre requête. La i-ème ligne contient les indices des k vecteurs les plus proches du i-ème vecteur de la requête.

* une matrice D des distances, de taille (n, k) également, où la i-ème ligne contient les distances (au carré) des k vecteurs les plus proches du i-ème vecteur de la requête.

In [42]:
#  Requête exemple (simule un utilisateur)
query = "où se situe la Gaule ?"
query_embedding = model.encode([query])

# Recherche du chunk le plus similaire (k=1)
D, I = index.search(query_embedding, k=1)

print("Résultat de la requête :", query)
print(f"Top match (indice FAISS): {I[0][0]}, Distance: {D[0][0]:.3f}")
print(f"Matrice I : {I}")
print("Chunk retrouvé :", chunks[I[0][0]])


Résultat de la requête : où se situe la Gaule ?
Top match (indice FAISS): 3, Distance: 0.825
Matrice I : [[3]]
Chunk retrouvé : surtout des rennes dont il utilise tous les éléments. (La peau pour les 
vêtements ou les cabanes, la graisse pour les lampes, les dents en 
collier…) 
 
*C’est un artisan capable de fabriquer de nombreux outils en 
taillant des pierres ou de l’os.  
 
*Nomade : adj : qui ne vit pas toujours au même endroit.  
 
 
 
 
 
4) Qui sont les Gaulois ? 
 
*On parle des Gaulois à partir de 600 ans avant JC.  
*La Gaule est constituée de nombreux peuples qui se 
font souvent la guerre.


In [45]:
# Recherche des deux chunks les plus similaires (k=2)
D, I = index.search(query_embedding, k=2)

print("Résultat de la requête :", query)
print(f"Top match (indice FAISS): {I[0][0]}, Distance: {D[0][0]:.3f}\n")
print(f"Matrice I : {I}")
print("\n")
print(I[0]) # numpy array
print(type(I[0]))
print(I[0].tolist())
print("Chunks retrouvés :")
for x in I[0].tolist():
    print("index :", x, "\n", chunks[x])
    print("------")


Résultat de la requête : où se situe la Gaule ?
Top match (indice FAISS): 3, Distance: 0.825

Matrice I : [[3 6]]


[3 6]
<class 'numpy.ndarray'>
[3, 6]
Chunks retrouvés :
index : 3 
 surtout des rennes dont il utilise tous les éléments. (La peau pour les 
vêtements ou les cabanes, la graisse pour les lampes, les dents en 
collier…) 
 
*C’est un artisan capable de fabriquer de nombreux outils en 
taillant des pierres ou de l’os.  
 
*Nomade : adj : qui ne vit pas toujours au même endroit.  
 
 
 
 
 
4) Qui sont les Gaulois ? 
 
*On parle des Gaulois à partir de 600 ans avant JC.  
*La Gaule est constituée de nombreux peuples qui se 
font souvent la guerre.
------
index : 6 
 peuples venus de l’est . Ils s’ installent en Gaule et 
l’empire romain disparaît  en 476 . C’est la fin de 
l’Antiquité et le début du Moyen Âge. 
 
*Les Francs, emmenés par Clovis conquièrent une grande partie de la 
Gaule. Clovis devient chrétien en se faisant baptiser en 496.
7) Charlemagne et les Carolingiens

### Test à partir de chunks sous la forme de "Langchain documents"

In [46]:
# Charger le modèle embedding avec Langchain
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

documents = [Document(page_content=chunk, metadata={"chunk_id": i}) for i, chunk in enumerate(chunks)]



In [47]:
documents

[Document(metadata={'chunk_id': 0}, page_content='LLeeççoonnss  dd’’hhiissttooiirree    CCMM11  \n \n \n1)  Qu’est-ce que l’Histoire ? \n \n*L’Histoire est l’étude de notre passé  pour mieux \ncomprendre notre vie aujourd’hui. \n \n *Pour découvrir notre passé, les historiens font des fouilles \narchéologiques, étudient des objets, des documents, des récits…  \n*Ils représentent le temps par une ligne graduée  : c’est la frise \nchronologique.  \n \n*Avant l’invention de l’écriture, c’est la Préhistoire , ensuite vient \nl’Histoire.'),
 Document(metadata={'chunk_id': 1}, page_content='l’Histoire.  \n*L’Histoire de France est divisée en 5 périodes : \nl’Antiquité – le Moyen Âge – les Temps Modernes – le XIX ème \nsiècle – le XXème siècle. \n \n \n \n \n2) Des traces du passé : les grottes ornées \n \n*En 1940, 4 enfants découvrent une grotte recouverte \nde peintures  : des taureaux, des cerfs, des chevaux…  : \nla grotte de Lascaux.  En datant les objets trouvés dans la grotte on \nsai

In [48]:
print("taille de documents :", len(documents))

taille de documents : 27


In [49]:
# Création de l’index FAISS dans LangChain
vectorstore = FAISS.from_documents(documents, embedding_model)

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [50]:
vectorstore

In [51]:
# Test de recherche
query = "Qui est Napoléon ?"
results = vectorstore.similarity_search(query, k=1)

print("Résultat le plus pertinent :")
print(results[0].page_content)

Résultat le plus pertinent :
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les lycées…


#### Autre façon de rechercher, en instanciant un "retriever"
cf https://python.langchain.com/docs/how_to/vectorstore_retriever/

* Il est possible de spécifier un "search_type" parmi les 3 modalités : 'similarity', 'similarity_score_threshold', 'mmr'. 

Par défaut (si aucun type_search n'est renseigné), c'est le "similarity" search qui est utilisé : It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs.

MMR = maximal marginal relevance. It selects examples based on a combination of which examples are most similar to the inputs, while also optimizing for diversity. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

Similarity score threshold :  threshold documents output by the retriever by similarity score

* On peut également spécifier des search_kwargs pour paramétrer le retriever (ex : nombre de résultats renvoyés)

In [60]:
# Différentes façons de rechercher dans la VectorDB

retrievers = {
    "retriever_similarity": vectorstore.as_retriever(
        search_type = "similarity",
        #search_kwargs = {"k": 1},
    ) ,
    "retriever_similarity_score_threshold": vectorstore.as_retriever(
        search_type = "similarity_score_threshold",
        search_kwargs = {'score_threshold': 0.48},
    ),
    "retriever_mmr": vectorstore.as_retriever(
        search_type = "mmr",
    ) 
}

In [61]:
for name, retriever in retrievers.items():
    print("--- Résultats pour le type de recherche :", name, "---")
    relevant_docs = retriever.invoke("Qui est Napoléon ?")
    print(type(relevant_docs))
    print(len(relevant_docs))
    print(relevant_docs)
    print("="*60)

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
No relevant docs were retrieved using the relevance score threshold 0.48


--- Résultats pour le type de recherche : retriever_similarity ---
<class 'list'>
4
[Document(id='6affccb2-6576-4527-943c-d7de32758ab6', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui donne tous les \npouvoirs.  \n \n*Il crée les départements dirigés par un préfet, la banque de France, \nles lycées…'), Document(id='40bb782c-3551-460c-b30c-9379fe08b1a6', metadata={'chunk_id': 25}, page_content='*Napoléon perd la bataille de Waterloo contre les Anglais et abdique \ndéfinitivement en 1815.    \n \n*Abdiquer : renoncer au pouvoir\n22) Les apports de la Révolution et de l’Empire \n \n\uf0b7 La Révolution et l’Empire ont permis de créer une \nadministration 

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Pour le similarity score threshold, il faut jouer sur le seuil (l'augmenter pour être plus restrictif). Dans l'exemple ci-dessus, si on spécifie un seuil de 30% de similarité, le nombre de chunks pertinents est de 4. Si on l'augmente à 35%, seuls deux documents sont pertinents. S'il est au-delà de 48% alors aucun document n'est considéré comme pertinent.

Le MMR renvoie le 1er résultat identique aux deux autres méthodes, mais pas les suivants qui sont d'ailleurs éloignés du sujet de la question. Les trois résultats suivants parlent de François Ier, le Moyen-Âge, et Saint Louis !

### Enregistrer la FAISS VectorDB dans un répertoire local

https://medium.com/@amrita.thakur/understanding-faiss-vector-store-and-its-advantages-cdc7b54afe47

In [62]:
db_path = "faiss_db" 

# Si le répertoire de la FAISS VectorDB existe déjà, le supprimer pour pouvoir en recréer un nouveau
if os.path.exists(db_path):
    shutil.rmtree(db_path)


In [63]:
# Enregistrement de la DB
vectorstore.save_local(db_path)

In [65]:
# Charger la FAISS VectorDB à partir d'un répertoire local
# Pour le chargement de la VectorDB, utiliser le même modèle d'embeddings que celui utilisé pour sa création
vector_store = FAISS.load_local(db_path, embedding_model, allow_dangerous_deserialization = True)


# Chroma DB

version client éphémère : https://docs.trychroma.com/docs/overview/getting-started
It starts a Chroma server in-memory, so any data you ingest will be lost when your program terminates. 

version client persistant : https://docs.trychroma.com/docs/run-chroma/persistent-client
You can configure Chroma to save and load the database from your local machine, using the PersistentClient.

https://python.langchain.com/docs/integrations/vectorstores/chroma/

In [72]:
chroma_db_path = "chroma_db"

In [73]:
chroma_db = Chroma.from_documents(
    documents, 
    embedding_model, 
    persist_directory = chroma_db_path)

In [74]:
# Test de recherche
query = "Qui est Napoléon ?"

# Récupère les chunks les plus pertinents associés à la requête
# Exemple avec une recherche basée sur un score de similarité
retriever = chroma_db.as_retriever(
    search_type = "similarity_score_threshold",
    search_kwargs = {"k": 3, "score_threshold": 0.2},
)

relevant_docs = retriever.invoke(query)

/usr/local/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [75]:
# le retriever retourne une liste de documents
relevant_docs

[Document(id='0ad561de-e278-4f51-9e9a-4874279d9dae', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui donne tous les \npouvoirs.  \n \n*Il crée les départements dirigés par un préfet, la banque de France, \nles lycées…'),
 Document(id='70d38bf3-3470-4a46-a1b6-1d4e9e5639d3', metadata={'chunk_id': 23}, page_content='autre gouvernement est mis en place.  \n \n*Il sera renversé par Napoléon le 18 brumaire.   \n \n  *Constitution : texte qui précise comment un pays est dirigé.\n21) Le Consulat et l’Empire (1799-1815) \n \n*Le général Bonaparte remporte de nombreuses victoires \nmilitaires, comme Austerlitz.  \n \n*Il  adopte une nouvelle constitution qui lui 

In [76]:
print("Nombre de chunks pertinents :", len(relevant_docs))
    
# Affiche les résultats pertinents avec les métadonnées associées
print("\n--- Documents les plus pertinents ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"Document {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"chunk_id : {doc.metadata.get('chunk_id')}\n")

Nombre de chunks pertinents : 3

--- Documents les plus pertinents ---
Document 1:
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les lycées…

chunk_id : 23

Document 2:
autre gouvernement est mis en place.  
 
*Il sera renversé par Napoléon le 18 brumaire.   
 
  *Constitution : texte qui précise comment un pays est dirigé.
21) Le Consulat et l’Empire (1799-1815) 
 
*Le général Bonaparte remporte de nombreuses victoires 
militaires, comme Austerlitz.  
 
*Il  adopte une nouvelle constitution qui lui donne tous les 
pouvoirs.  
 
*Il crée les départements dirigés par un préfet, la banque de France, 
les